## Erzeugung von Testdaten mittels *Ragas*

In dieser Übung geht es darum, mittels [Ragas](https://docs.ragas.io/) automatisch Testfragen zu den Wahlprogrammen der Parteien zu erstellen. 
Dabei wollen wir zwei Aspekte anpassen:

- Wir wollen konkrete *Personas* für die Testragen vorgeben,
- die Fragen sollen auf Deutsch erstellt werden.

In [ ]:
!pip install  --upgrade langchain langchain-community langchain_chroma langchain_openai langchain_unstructured  unstructured[pdf] chromadb ragas

In [15]:
import chromadb
import os
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_unstructured.document_loaders import UnstructuredLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements
from unstructured.documents.elements import Image

Wir verwenden wieder die `.env` Datei aus dem letzten Praktikum.

In [18]:
load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#RAGAS_APP_TOKEN = os.getenv("RAGAS_APP_TOKEN")
DATABASE_PATH = "./chroma/"
EMBEDDING_MODEL = "text-embedding-ada-002"

**Bitte ändern Sie das voreingestellte Modell *nicht*!!**

In [19]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

### Laden der Parteiprogramme

Wir laden zunächst die Dateien. Prüfen Sie anschließend, ob alle Dokumente geladen wurden.

In [16]:
from langchain_community.document_loaders import DirectoryLoader

path = "./data/"
loader = DirectoryLoader(path, glob="**/*.pdf")
docs = loader.load()

### Generieren eines Testdatensatzes mit Default-Einstellungen

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

### Bewertung des Testsets

Wandeln Sie das Testset in ein Pandas Dataframe und schauen Sie sich die Fragen an.

- Wie viele sind auf Deutsch?
- Welche Haltung/Einstellung würden Sie bei einem menschlichen Fragesteller jeweils vermuten?
- Sind die Fragen "neutral" oder kommt die Haltung einer Partei zum Ausdruck? 

### Customizing der Testset-Generierung

Fügen Sie weitere *Personas* hinzu, die unterschiedliche Bevölkerungsgruppen und politische Ansichten wiederspiegeln.

In [ ]:
from ragas.testset.persona import Persona

personas = [
    Persona(
        name="Dorfbewohner aus einem ländlichen Gebiet",
        role_description="Ein Bürger aus einem ländlichen Gebiet, der sich um die Infrastruktur auf dem Land sorgt und im Alltag auf den Individualverkehr angewiesen ist.",
    ),
]

In [21]:
generator = TestsetGenerator(
    llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas
)

In [22]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

Schauen Sie sich die erzeugte `distribution` an. Was fällt Ihnen auf?

Wir passen nun die Sprache der Testerzeugung an.

In [24]:
for query, _ in distribution:
    prompts = await query.adapt_prompts("deutsch", llm=generator_llm)
    query.set_prompts(**prompts)

Was hat sich geändert?

In [ ]:
dataset = generator.generate_with_langchain_docs(
    docs,
    testset_size=5,
    query_distribution=distribution,
)

Vergleichen Sie das neu erzeugte Testset:

- Inwiefern spiegeln sich die Personas in den Fragen wieder?
- Wie viele Fragen sind auf Deutsch?
- Vergleichen Sie Ihre Personas und Testsets mit denen Ihrer Kommilitonen. Wie beurteilen Sie die Qualität der erstellten Testfragen?  